In [1]:
from EEG2CodeKeras import (basearchi,
                           basearchitest_batchnorm,
                           basearchi_patchembedding,
                           basearchi_patchembeddingdilation,
                           trueVanilliaEEG2Code,
                           vanilliaEEG2Code,
                           vanilliaEEG2Code2,
                           EEGnet_Inception)
from _utils import make_preds_accumul_aggresive, make_preds_pvalue

from utils import prepare_data, get_BVEP_data, balance



from sklearn.model_selection import train_test_split
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import balanced_accuracy_score,confusion_matrix
from sklearn.cross_decomposition import CCA
import keras


from pyriemann.estimation import XdawnCovariances, Xdawn
from sklearn.manifold import TSNE
from pyriemann.tangentspace import TangentSpace
from pyriemann.estimation import Covariances
from pyriemann.utils.distance import distance_riemann, distance
from pyriemann.utils.utils import check_weights
from pyriemann.utils.base import powm
from pyriemann.tangentspace import TangentSpace
from mne.decoding import Vectorizer
from pyriemann.transfer import (
    decode_domains,
    encode_domains,
    TLCenter,
    TLStretch,
    TLRotate,
)

from tensorflow import keras
from collections import OrderedDict
import tensorflow as tf
import mne
import time
import os

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import sys
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt



c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to en

In [2]:
from moabb.paradigms import CVEP
from SPDNet.tensorflow.spd_net_2_tensorflow import SPDNet_AJD
from SPDNet.tensorflow.spd_net_tensorflow import SPDNet_Tensorflow
# from SPDNet.tensorflow.optimizer import riemannian_adam
from SPDNet.torch.optimizers import riemannian_adam as torch_riemannian_adam
from SPDNet.torch.spd_net_bn_torch import SPDNetBN_Torch, SPDNetBN_Module,CNNSPDNetBN_Module
from DNorm_CLF.DNorm_SPD.DNorm_SPD import BNSPD_Net
from sklearn.model_selection import GridSearchCV

sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\moabb\\moabb\\datasets")
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\moabb\\moabb\\paradigms")
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Scripts\\SPDNet")
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\riemannian_tSNE")
from R_TSNE import R_TSNE
from castillos2023 import CasitllosCVEP100,CasitllosCVEP40,CasitllosBurstVEP100,CasitllosBurstVEP40
from torch.utils.data import DataLoader, TensorDataset


Instructions for updating:
experimental_compile is deprecated, use jit_compile instead


# Parameters initialization

In [3]:
window_size=0.25
n_class=4
sfreq = 500
fps = 60

## Load the data

In [4]:
subjects = [1,2,3,4,5,6,7,8,9,10,11,12]
# subjects = [1,2,3]
n_channels = 32
on_frame = False
tospd = False
recenter = True
normalise = False
if on_frame:
    freq = fps
else:
    freq = sfreq

raw_data,labels,codes,labels_codes = get_BVEP_data(subjects,on_frame)
X_parent,Y_parent,domains_parent = prepare_data(subjects,raw_data,labels,on_frame,tospd,recenter,codes=codes)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 50 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 49.90, 50.10 Hz: -6.02, -6.02 dB

Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 50 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 49.90, 50.10 Hz: -6.02, -6.02 dB

Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 50 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandstop zero-phase (two-pas

## Train test the new model

In [5]:
n_cal = 7
n_class = 4
nb_fold = 1
spdbn_accuracy_code_perso = np.zeros((nb_fold,12))
spdbn_tps_train_code_perso = np.zeros((nb_fold,12))
spdbn_tps_test_code_perso = np.zeros((nb_fold,12))
spdbn_accuracy_perso = np.zeros((nb_fold,12))

for k in range(nb_fold):
    for i in range(12):
        print("TL to the participant : ", i)
        X = X_parent.copy()
        Y = Y_parent.copy()
        domains = domains_parent.copy()
        nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*freq)

        X_train = X[i][:nb_sample_cal]
        Y_train = Y[i][:nb_sample_cal]
        X_test = X[i][nb_sample_cal:]
        Y_test = Y[i][nb_sample_cal:]
        labels_code_test = labels_codes[i][(n_class*n_cal):]

        print(X_train.shape)
        print(X_test.shape)
        # X_std = X_train.std(axis=0)
        # X_train /= X_std + 1e-8
        # X_std = X_test.std(axis=0)
        # X_test /= X_std + 1e-8

        print("balancing the number of ones and zeros")
        X_train, Y_train, domains_train = balance(X_train,Y_train,domains[i][:nb_sample_cal])

        print("Creating the different pipelines")
        clf = CNNSPDNetBN_Module(32,0.25)

        print("Fitting")
        start = time.time()
        weight_decay = 1e-4
        
        x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)


        # Convert data into PyTorch tensors
        X_train_tensor = torch.tensor(x_train, dtype=torch.float64)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_val_tensor = torch.tensor(x_val, dtype=torch.float64)
        y_val_tensor = torch.tensor(y_val, dtype=torch.long)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float64)
        y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

        # Create DataLoader for train, validation, and test sets
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Define loss function and optimizer
        criterion = torch.nn.CrossEntropyLoss().float()
        optimizer = torch_riemannian_adam.RiemannianAdam(clf.parameters(), lr=0.001)

        # Train the model
        num_epochs = 20
        for epoch in range(num_epochs):
            running_loss = 0.0
            for inputs, labels in train_dataloader:
                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = clf(inputs)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}")

        print("Training finished!")
        spdbn_tps_train_code_perso[k][i] = time.time() - start

        # Validation
        clf.eval()
        val_correct = 0
        y_pred = []
        y_true = []
        with torch.no_grad():
            for inputs, labels in val_dataloader:
                outputs = clf(inputs)
                _, predicted = torch.max(outputs, 1)
                y_pred.append(np.array(predicted))
                y_true.append(np.array(labels)) 
                val_correct += (predicted == labels).sum().item()

        val_accuracy = balanced_accuracy_score(np.concatenate(y_true),np.concatenate(y_pred))
        print(f"Validation Accuracy: {val_accuracy}")

        # Testing
        start = time.time()
        test_correct = 0
        y_pred= []
        y_true = []
        with torch.no_grad():
            for inputs, labels in test_dataloader:
                outputs = clf(inputs)
                _, predicted = torch.max(outputs, 1)
                y_pred.append(np.array(predicted))
                y_true.append(np.array(labels)) 
                test_correct += (predicted == labels).sum().item()
                
        test_accuracy = balanced_accuracy_score(np.concatenate(y_true),np.concatenate(y_pred))
        print(f"Test Accuracy: {test_accuracy}")
        
        print("getting accuracy of participant ", i)
        y_pred = np.concatenate(y_pred)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
        y_test_norm = np.array([0 if y == 0 else 1 for y in np.concatenate(y_true)])

        tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        spdbn_accuracy_perso[k][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=freq, consecutive=50, window_size=window_size
        )
        spdbn_tps_test_code_perso[k][i] = time.time() - start
        spdbn_accuracy_code_perso[k][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
        keras.backend.clear_session()

spdbn_accuracy_perso = np.mean(spdbn_accuracy_perso,axis=0)
spdbn_tps_train_code_perso = np.mean(spdbn_tps_train_code_perso,axis=0)
spdbn_tps_test_code_perso = np.mean(spdbn_tps_test_code_perso,axis=0)
spdbn_accuracy_code_perso = np.mean(spdbn_accuracy_code_perso,axis=0)

print(spdbn_accuracy_perso)
print(spdbn_tps_train_code_perso)
print(spdbn_tps_test_code_perso)
print(spdbn_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNNSPD/score/WO_score",spdbn_accuracy_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNNSPD/score_code/WO_score_code",spdbn_tps_train_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNNSPD/temps_code/WO_tps_train_code",spdbn_tps_test_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNNSPD/temps_code/WO_tps_test_code",spdbn_accuracy_code_perso)

TL to the participant :  0
(27300, 32, 125)
(31200, 32, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6944585834729912
Epoch 2, Loss: 0.5538793123006814
Epoch 3, Loss: 0.4291010735444135
Epoch 4, Loss: 0.35639989878538864
Epoch 5, Loss: 0.31818655786797156
Epoch 6, Loss: 0.2979181310311357
Epoch 7, Loss: 0.277120313782002
Epoch 8, Loss: 0.2668423019211832
Epoch 9, Loss: 0.2549846105058366
Epoch 10, Loss: 0.24095147769406752
Epoch 11, Loss: 0.2326553807756013
Epoch 12, Loss: 0.22620270569760453
Epoch 13, Loss: 0.21972339399086088
Epoch 14, Loss: 0.21952193174158685
Epoch 15, Loss: 0.21610366727937289
Epoch 16, Loss: 0.2024741565439152
Epoch 17, Loss: 0.20901847885702446
Epoch 18, Loss: 0.19118114578634288
Epoch 19, Loss: 0.19096887021093514
Epoch 20, Loss: 0.19683055049177375
Training finished!
Validation Accuracy: 0.9236740210403273
Test Accuracy: 0.8225257019994665
getting accuracy of participant  0

TL to the participant :  1
(2

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  2
(27300, 32, 125)
(31200, 32, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6089175089580071
Epoch 2, Loss: 0.40532228520371383
Epoch 3, Loss: 0.33366999959739896
Epoch 4, Loss: 0.3028602772043272
Epoch 5, Loss: 0.28264925289253684
Epoch 6, Loss: 0.2644562651968652
Epoch 7, Loss: 0.25492864772315604
Epoch 8, Loss: 0.2457273748200802
Epoch 9, Loss: 0.2392385746082766
Epoch 10, Loss: 0.23726030278736057
Epoch 11, Loss: 0.226144069289147
Epoch 12, Loss: 0.2221934774891392
Epoch 13, Loss: 0.2208465063164683
Epoch 14, Loss: 0.2123982309048172
Epoch 15, Loss: 0.21529391809145876
Epoch 16, Loss: 0.2088198176569065
Epoch 17, Loss: 0.20835771683859108
Epoch 18, Loss: 0.20391879441323518
Epoch 19, Loss: 0.20131233226748427
Epoch 20, Loss: 0.19165491821565006
Training finished!
Validation Accuracy: 0.9329609449279602
Test Accuracy: 0.8088148038699828
getting accuracy of participant  2


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  3
(27300, 32, 125)
(31200, 32, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6387841367938223
Epoch 2, Loss: 0.4986293820271304
Epoch 3, Loss: 0.4176959047531361
Epoch 4, Loss: 0.3677853627625593
Epoch 5, Loss: 0.33602066822302906
Epoch 6, Loss: 0.3138773883975892
Epoch 7, Loss: 0.293397057568426
Epoch 8, Loss: 0.2869163280829963
Epoch 9, Loss: 0.265244901502658
Epoch 10, Loss: 0.25920052826156814
Epoch 11, Loss: 0.25690051368139827
Epoch 12, Loss: 0.23648418452797934
Epoch 13, Loss: 0.23358811821060335
Epoch 14, Loss: 0.22651925583999202
Epoch 15, Loss: 0.22815976908480923
Epoch 16, Loss: 0.21046135010391814
Epoch 17, Loss: 0.20905428431570455
Epoch 18, Loss: 0.2166762476149154
Epoch 19, Loss: 0.20687310845027718
Epoch 20, Loss: 0.2023017041532847
Training finished!
Validation Accuracy: 0.8677083333333333
Test Accuracy: 0.7434163232940061
getting accuracy of participant  3
TL to the participant :  4
(2730

...

TL to the participant :  9
(27300, 32, 125)
(31200, 32, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.4755443377614996
Epoch 2, Loss: 0.23896264601007958
Epoch 3, Loss: 0.17894376774903886
Epoch 4, Loss: 0.14666856194228448
Epoch 5, Loss: 0.1299241162150527
Epoch 6, Loss: 0.12024325813908902
Epoch 7, Loss: 0.11076336739179644
Epoch 8, Loss: 0.10355339091992877
Epoch 9, Loss: 0.09565281163319313
Epoch 10, Loss: 0.08918611420835314
Epoch 11, Loss: 0.09117001838952837
Epoch 12, Loss: 0.08363024666469683
Epoch 13, Loss: 0.0863496537779668
Epoch 14, Loss: 0.08233874957994113
Epoch 15, Loss: 0.07610422070448952
Epoch 16, Loss: 0.07776269514310578
Epoch 17, Loss: 0.08258037855050442
Epoch 18, Loss: 0.0779208525217385
Epoch 19, Loss: 0.07741501022307638
Epoch 20, Loss: 0.07389803608285202
Training finished!
Validation Accuracy: 0.9818032809339658
Test Accuracy: 0.9572688626459951
getting accuracy of participant  9
TL to the participant : 

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  11
(27300, 32, 125)
(31200, 32, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6988589993596271
Epoch 2, Loss: 0.6316084550181587
Epoch 3, Loss: 0.5676488655577747
Epoch 4, Loss: 0.5178162628427988
Epoch 5, Loss: 0.4723283810763644
Epoch 6, Loss: 0.4444703937010373
Epoch 7, Loss: 0.4204618234359164
Epoch 8, Loss: 0.4007349701106623
Epoch 9, Loss: 0.39097634345085813
Epoch 10, Loss: 0.37553006662433996
Epoch 11, Loss: 0.3559364430087989
Epoch 12, Loss: 0.34972863752088507
Epoch 13, Loss: 0.32834511773004044
Epoch 14, Loss: 0.3240028661197094
Epoch 15, Loss: 0.3167406407839553
Epoch 16, Loss: 0.31306491096586736
Epoch 17, Loss: 0.30993571644063683
Epoch 18, Loss: 0.3074419455129964
Epoch 19, Loss: 0.3008260238105924
Epoch 20, Loss: 0.2966516059031771
Training finished!
Validation Accuracy: 0.9086363947674817
Test Accuracy: 0.638233009537855
getting accuracy of participant  11


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0.8225257  0.63297566 0.8088148  0.74341632 0.81250973 0.57244673
 0.7637732  0.86845488 0.66102645 0.95726886 0.73309928 0.63823301]
[183.87130761 183.95863271 184.72878742 135.65179062 122.35824776
 124.8109901  162.6724081  129.6378696  225.1806252  250.03609228
 221.30211592 240.05810714]
[17.17922807 17.14903259 16.96914411 10.0809536  10.59220839 10.04147959
 17.01987076 10.97256446 26.2461462  19.94112277 20.36900353 25.13798428]
[0.94 0.75 0.97 0.91 1.   0.59 0.91 0.94 0.72 1.   0.91 0.72]


## LOO the new model

In [5]:
n_cal = 7
n_class = 4
nb_fold = 1
spdbn_accuracy_code_perso = np.zeros((nb_fold,12))
spdbn_tps_train_code_perso = np.zeros((nb_fold,12))
spdbn_tps_test_code_perso = np.zeros((nb_fold,12))
spdbn_accuracy_perso = np.zeros((nb_fold,12))

for k in range(nb_fold):
    for i in range(12):
        print("TL to the participant : ", i)
        ind2take = [j for j in range(12) if j!=i]
        X = X_parent.copy()
        Y = Y_parent.copy()
        domains = domains_parent.copy()
        
        X_train = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
        Y_train = np.concatenate(Y[ind2take]).reshape(-1)
        domains_train = domains[ind2take].reshape(-1)
        X_test = X[i]
        Y_test = Y[i]
        labels_code_test = labels_codes[i]

        print(X_train.shape)
        print(X_test.shape)
        # X_std = X_train.std(axis=0)
        # X_train /= X_std + 1e-8
        # X_std = X_test.std(axis=0)
        # X_test /= X_std + 1e-8

        print("balancing the number of ones and zeros")
        X_train, Y_train, domains_train = balance(X_train,Y_train,domains_train)

        print("Creating the different pipelines")
        clf = CNNSPDNetBN_Module(32,0.25)

        print("Fitting")
        start = time.time()
        weight_decay = 1e-4
        
        x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)


        # Convert data into PyTorch tensors
        X_train_tensor = torch.tensor(x_train, dtype=torch.float64)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_val_tensor = torch.tensor(x_val, dtype=torch.float64)
        y_val_tensor = torch.tensor(y_val, dtype=torch.long)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float64)
        y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

        # Create DataLoader for train, validation, and test sets
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Define loss function and optimizer
        criterion = torch.nn.CrossEntropyLoss().float()
        optimizer = torch_riemannian_adam.RiemannianAdam(clf.parameters(), lr=0.001)

        # Train the model
        num_epochs = 20
        for epoch in range(num_epochs):
            running_loss = 0.0
            for inputs, labels in train_dataloader:
                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = clf(inputs)
                loss = criterion(outputs, labels)

                # Backward pass and optimize
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}")

        print("Training finished!")
        spdbn_tps_train_code_perso[k][i] = time.time() - start

        # Validation
        clf.eval()
        val_correct = 0
        y_pred = []
        y_true = []
        with torch.no_grad():
            for inputs, labels in val_dataloader:
                outputs = clf(inputs)
                _, predicted = torch.max(outputs, 1)
                y_pred.append(np.array(predicted))
                y_true.append(np.array(labels)) 
                val_correct += (predicted == labels).sum().item()

        val_accuracy = balanced_accuracy_score(np.concatenate(y_true),np.concatenate(y_pred))
        print(f"Validation Accuracy: {val_accuracy}")

        # Testing
        start = time.time()
        test_correct = 0
        y_pred= []
        y_true = []
        with torch.no_grad():
            for inputs, labels in test_dataloader:
                outputs = clf(inputs)
                _, predicted = torch.max(outputs, 1)
                y_pred.append(np.array(predicted))
                y_true.append(np.array(labels)) 
                test_correct += (predicted == labels).sum().item()
                
        test_accuracy = balanced_accuracy_score(np.concatenate(y_true),np.concatenate(y_pred))
        print(f"Test Accuracy: {test_accuracy}")
        
        print("getting accuracy of participant ", i)
        y_pred = np.concatenate(y_pred)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
        y_test_norm = np.array([0 if y == 0 else 1 for y in np.concatenate(y_true)])

        tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        spdbn_accuracy_perso[k][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=freq, consecutive=50, window_size=window_size
        )
        spdbn_tps_test_code_perso[k][i] = time.time() - start
        spdbn_accuracy_code_perso[k][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
        keras.backend.clear_session()

spdbn_accuracy_perso = np.mean(spdbn_accuracy_perso,axis=0)
spdbn_tps_train_code_perso = np.mean(spdbn_tps_train_code_perso,axis=0)
spdbn_tps_test_code_perso = np.mean(spdbn_tps_test_code_perso,axis=0)
spdbn_accuracy_code_perso = np.mean(spdbn_accuracy_code_perso,axis=0)

print(spdbn_accuracy_perso)
print(spdbn_tps_train_code_perso)
print(spdbn_tps_test_code_perso)
print(spdbn_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNNSPD/score/LOO_score",spdbn_accuracy_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNNSPD/score_code/LOO_score_code",spdbn_tps_train_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNNSPD/temps_code/LOO_tps_train_code",spdbn_tps_test_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNNSPD/temps_code/LOO_tps_test_code",spdbn_accuracy_code_perso)

TL to the participant :  0
(643500, 32, 125)
(58500, 32, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5277659098280294
Epoch 2, Loss: 0.4841184514596053
Epoch 3, Loss: 0.47832595402633277
Epoch 4, Loss: 0.47431327800346434
Epoch 5, Loss: 0.47307354526412176
Epoch 6, Loss: 0.46983986987853216
Epoch 7, Loss: 0.469540729996048
Epoch 8, Loss: 0.467504229511115
Epoch 9, Loss: 0.466709942542084
Epoch 10, Loss: 0.46533458289430524
Epoch 11, Loss: 0.4654226899812408
Epoch 12, Loss: 0.4646671213304908
Epoch 13, Loss: 0.46413536576486686
Epoch 14, Loss: 0.4640318215520198
Epoch 15, Loss: 0.46360225276941125
Epoch 16, Loss: 0.463363304505864
Epoch 17, Loss: 0.4618279601328525
Epoch 18, Loss: 0.46237815643406865
Epoch 19, Loss: 0.4618663814736231
Epoch 20, Loss: 0.4623452462479358
Training finished!
Validation Accuracy: 0.8076190407044691
Test Accuracy: 0.8263275429339341
getting accuracy of participant  0

TL to the participant :  1
(643500

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  2
(643500, 32, 125)
(58500, 32, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5203625199106414
Epoch 2, Loss: 0.48177368665286074
Epoch 3, Loss: 0.4749256106405472
Epoch 4, Loss: 0.47272641160351186
Epoch 5, Loss: 0.47011318851483624
Epoch 6, Loss: 0.4685026138482137
Epoch 7, Loss: 0.46778867794458856
Epoch 8, Loss: 0.4657345656287458
Epoch 9, Loss: 0.4652122970461063
